In [1]:
def load_rois_atlas():
    """
    Loads ROIs from an atlas.

    Parameters
    ----------
    atlas_name : str
        The name of the atlas.
    surf_size : str
        Size of the surface, either '59k' or '170k'.
    return_hemis : bool, optional
        If True, returns ROIs for both hemispheres separately. If False, returns combined ROIs.
        Default is False.
    rois : list of str, optional
        List of ROIs you want to extract. If None, all ROIs are returned. 
        Default is None.
    mask : bool, optional
        If True, returns the ROI masks. If False, returns the indices where the masks are True.
        Default is True.
    path_to_atlas : str, optional
        Path to the directory containing the atlas data. If not provided, the function looks for the atlas 
        data in the default directory.

    Returns
    -------
    rois_masks : dict or tuple of dicts
        A dictionary or tuple of dictionaries where the keys represent the ROIs and the values correspond 
        to the respective masks for each hemisphere.

    Raises
    ------
    ValueError
        If 'surf_size' is not '59k' or '170k'.
    """
    import os
    import numpy as np
    
    # Validating surf_size
    if surf_size not in ['59k', '170k']:
        raise ValueError("Invalid value for 'surf_size'. It should be either '59k' or '170k'.")
    
    # Loading data from the specified path or default directory
    if path_to_atlas:
        data_path = path_to_atlas
    else:    
        atlas_dir = os.path.abspath(os.path.join(os.path.dirname(__file__), "../atlas/"))
        data_path = atlas_dir
    
    if return_hemis:
        filename_L = "{}_atlas_rois_{}_hemi-L.npz".format(atlas_name, surf_size)
        data_L = np.load(os.path.join(data_path, filename_L))
        rois_dict_L = dict(data_L)
        
        filename_R = "{}_atlas_rois_{}_hemi-R.npz".format(atlas_name, surf_size)
        data_R = np.load(os.path.join(data_path, filename_R))
        rois_dict_R = dict(data_R)
        
        # Handling the case where mask is False
        if not mask:
            rois_dict_L = {roi: np.where(rois_dict_L[roi])[0] for roi in rois_dict_L}
            rois_dict_R = {roi: np.where(rois_dict_R[roi])[0] for roi in rois_dict_R}
            
        # Filtering ROIs if rois is provided
        if rois is None:
            return rois_dict_L, rois_dict_R
        elif isinstance(rois, list):
            filtered_rois_L = {roi: rois_dict_L[roi] for roi in rois if roi in rois_dict_L}
            filtered_rois_R = {roi: rois_dict_R[roi] for roi in rois if roi in rois_dict_R}
            return filtered_rois_L, filtered_rois_R
        else:
            raise ValueError("Invalid value for 'rois'. It should be either None or a list of ROI names.")
    else: 
        filename = "{}_atlas_rois_{}.npz".format(atlas_name, surf_size)
        data = np.load(os.path.join(data_path, filename))
        rois_dict = dict(data)
        
        # Handling the case where mask is False
        if not mask:
            rois_dict = {roi: np.where(rois_dict[roi])[0] for roi in rois_dict}
            
        # Filtering ROIs if rois is provided
        if rois is None:
            return rois_dict
        elif isinstance(rois, list):
            filtered_rois = {roi: rois_dict[roi] for roi in rois if roi in rois_dict}
            return filtered_rois
        else:
            raise ValueError("Invalid value for 'rois'. It should be either None or a list of ROI names.")

In [25]:

def get_rois(subject, return_concat_hemis=False, return_hemi=None, rois=None, mask=True, atlas_name=None, surf_size=None, overlay_fn=None):
    """
    Accesses single hemisphere ROI masks for GIFTI and atlas ROI for CIFTI.

    Parameters
    ----------
    subject : str
        Subject name in the pycortex database.
    return_concat_hemis : bool, optional
        Indicates whether to return concatenated hemisphere ROIs. Defaults to False.
    return_hemi : str, optional
        Indicates which hemisphere's ROI masks to return. Can be 'hemi-L' for the left hemisphere or 'hemi-R' for the right hemisphere.
    rois : list of str, optional
        List of ROIs you want to extract.
    mask : bool, optional
        Indicates whether to mask the ROIs. Defaults to True.
    atlas_name : str, optional
        If atlas_name is not None, subject has to be a template subject (i.e., sub-170k).
        If provided, `surf_size` must also be specified.
    surf_size : str, optional
        The size in which you want the ROIs. It should be '59k' or '170k'. 
        Required if `atlas_name` is provided.
    overlay_fn : str, optional
        File name of the overlay file (e.g. 'overlay_rois-drawn.svg')

    Returns
    -------
    rois_masks : dict or tuple of dicts
        A dictionary or tuple of dictionaries containing the ROI masks.
        If `atlas_name` is provided, returns ROIs from the specified atlas.
        If `atlas_name` is None, returns subject-specific ROIs from pycortex.
        
        If `atlas_name` is provided:
        - If `return_concat_hemis` is True, returns a single dictionary with concatenated hemisphere ROIs.
        - If `return_hemi` is specified, returns ROIs for the specified hemisphere.
        - If neither `return_concat_hemis` nor `return_hemi` is specified, returns ROIs for both hemispheres in a tuple of dictionaries.
        
        If `atlas_name` is None:
        - If `return_concat_hemis` is True, returns a single dictionary with concatenated hemisphere ROIs.
        - If `return_hemi` is specified, returns ROIs for the specified hemisphere.
        - If neither `return_concat_hemis` nor `return_hemi` is specified, returns ROIs for both hemispheres in a tuple of dictionaries.

    Notes
    -----
    For both cases (`atlas_name` provided or not), ROI masks are represented as dictionaries where the keys represent the ROI names and 
    the values correspond to the respective masks for each hemisphere. Each mask is an array of vertex indices indicating the locations of the ROI on the cortical surface.
    """ 
    import cortex
    import numpy as np
    
    surfs = [cortex.polyutils.Surface(*d) for d in cortex.db.get_surf(subject, "flat")]
    surf_lh, surf_rh = surfs[0], surfs[1]
    lh_vert_num, rh_vert_num = surf_lh.pts.shape[0], surf_rh.pts.shape[0]

    # define overlay
    if overlay_fn == 'None': 
        overlay_file = None
    else:
        # define overlay path
        pycortex_config_file  = cortex.options.usercfg
        with open(pycortex_config_file, 'r') as fileIn:
            for line in fileIn:
                if 'filestore' in line:
                    db_path=line[10:-2]
        overlay_file = f"{db_path}/{subject}/{overlay_fn}"
    
    # get rois 
    if atlas_name :
        roi_verts = load_rois_atlas(atlas_name=atlas_name, 
                                    surf_size=surf_size,
                                    return_hemis=False,
                                    rois=rois, 
                                    mask=mask)
        
        if return_concat_hemis :
            return roi_verts
        
        
        elif return_hemi == 'hemi-L':
            roi_verts_L, roi_verts_R = load_rois_atlas(atlas_name=atlas_name, surf_size=surf_size, return_hemis=True, rois=rois, mask=mask)
            return roi_verts_L
        elif return_hemi == 'hemi-R':
            roi_verts_L, roi_verts_R = load_rois_atlas(atlas_name=atlas_name, surf_size=surf_size, return_hemis=True, rois=rois, mask=mask)
            return roi_verts_R
        else:
            roi_verts_L, roi_verts_R = load_rois_atlas(atlas_name=atlas_name, surf_size=surf_size, return_hemis=True, rois=rois, mask=mask)
            return roi_verts_L, roi_verts_R

    else:
        roi_verts = cortex.get_roi_verts(subject=subject, 
                                          roi=rois, 
                                          mask=True,
                                          overlay_file=overlay_file)
        rois_masks_L = {roi: data[:lh_vert_num] for roi, data in roi_verts.items()}
        rois_masks_R = {roi: data[-rh_vert_num:] for roi, data in roi_verts.items()}
        
        if mask==True:
            if return_concat_hemis :
                return roi_verts
            elif return_hemi == 'hemi-L':
                return rois_masks_L
            elif return_hemi == 'hemi-R':
                return rois_masks_R
            else:
                return rois_masks_L, rois_masks_R

        else:
            rois_idx_L = {roi: np.where(rois_masks_L[roi])[0] for roi in rois_masks_L}
            rois_idx_R = {roi: np.where(rois_masks_R[roi])[0] for roi in rois_masks_R}

            if return_concat_hemis :
                roi_verts = cortex.get_roi_verts(subject=subject, 
                                                 roi=rois, 
                                                 mask=False,
                                                 overlay_file=overlay_file)
                return roi_verts
            elif return_hemi == 'hemi-L':
                return rois_idx_L
            elif return_hemi == 'hemi-R':
                return rois_idx_R
            else:
                return rois_idx_L, rois_idx_R


In [26]:
get_rois(subject, return_concat_hemis=False, return_hemi='hemi-L', rois=None, mask=True, atlas_name=None, surf_size=None, overlay_fn=None)


Create new file: /Users/uriel/disks/meso_shared/RetinoMaps/derivatives/pp_data/cortex/db/sub-01/None


{}

In [14]:
rois_dict = cortex.get_roi_verts(subject='sub-01', mask=True, overlay_file='/Users/uriel/disks/meso_shared/RetinoMaps/derivatives/pp_data/cortex/db/sub-01/overlays_rois-drawn.svg')



In [16]:
rois_dict['V1'].shape

(459846,)

In [ ]:
def get_rois(subject, surf_format, atlas_name, rois, mask, return_concat_hemis, return_hemi):


    
    
    



    

subject : subject ID
surf_format : 170k, 59k, 91k, 32k, fsnative
atlas_name : mmp, group-mmp, drawn 
rois : []
mask : True/False
hemis : ['hemi-L', 'hemi-R']

In [19]:
subject = 'sub-01'

In [34]:
def get_rois(subject, surf_format, atlas_name, mask=True, rois=None, hemis=None):
    """
    Load ROI masks stored as .npz files from a pycortex subject database.

    The ROI files are expected to be dictionaries where:
        - keys are ROI names (str)
        - values are boolean numpy arrays
          (True = vertex belongs to the ROI, False otherwise)

    Depending on the arguments, the function can return:
        - full-brain ROI masks
        - left or right hemisphere ROI masks
        - both hemispheres simultaneously

    Parameters
    ----------
    subject : str
        Pycortex subject name.
        Must be:
            - 'sub-hcp59k' for surf_format in ['170k', '59k']
            - 'sub-hcp32k' for surf_format in ['91k', '32k']

    surf_format : str
        Surface format of the ROI masks.
        Allowed values are:
            ['170k', '59k', '91k', '32k', 'fsnative']

    atlas_name : str
        Name of the atlas used to define the ROIs.
        Allowed values are:
            ['mmp', 'group-mmp', 'drawn']

    mask : bool, optional (default=True)
        If True, return boolean masks.
        If False, return vertex indices (np.where(mask)[0]).
        Indices are relative to the requested space
        (full brain or hemisphere).

    rois : list of str or None, optional (default=None)
        List of ROI names to load.
        If None, all ROIs found in the file are returned.

    hemis : None, str, or list of str, optional (default=None)
        Hemisphere selection:
            - None : load full-brain ROIs
            - 'hemi-L' : load left hemisphere ROIs
            - 'hemi-R' : load right hemisphere ROIs
            - ['hemi-L', 'hemi-R'] : load both hemispheres

    Returns
    -------
    dict
        ROI masks or vertex indices, depending on input arguments.

    Notes
    -----
    Expected file naming conventions:

    - Full brain:
        {subject}_{surf_format}_rois-{atlas_name}.npz

    - Single hemisphere:
        {subject}_{surf_format}_hemi-L_rois-{atlas_name}.npz
        {subject}_{surf_format}_hemi-R_rois-{atlas_name}.npz

    All files must be located in:
        <pycortex_db>/<subject>/rois/
    """
    import cortex
    import numpy as np
    import os

    # --------------------------------------------------
    # Input checks
    # --------------------------------------------------
    if surf_format not in ['170k', '59k', '91k', '32k', 'fsnative']:
        raise ValueError("Invalid value for 'surf_format'.")
    if atlas_name not in ['mmp', 'group-mmp', 'drawn']:
        raise ValueError("Invalid value for 'atlas_name'.")
    if surf_format in ['170k', '59k'] and subject != 'sub-hcp59k':
        raise ValueError("For this format subject should be sub-hcp59k")
    if surf_format in ['91k', '32k'] and subject != 'sub-hcp32k':
        raise ValueError("For this format subject should be sub-hcp32k")

    # --------------------------------------------------
    # Paths
    # --------------------------------------------------
    db_dir = cortex.database.default_filestore
    rois_dir = os.path.join(db_dir, subject, 'rois')

    # --------------------------------------------------
    # Full brain
    # --------------------------------------------------
    if hemis is None:
        rois_dict_brain = dict(
            np.load(
                '{}/{}_{}_rois-{}.npz'.format(
                    rois_dir, subject, surf_format, atlas_name
                )
            )
        )

        if rois is not None:
            rois_dict_brain = {
                roi_name: roi_mask
                for roi_name, roi_mask in rois_dict_brain.items()
                if roi_name in rois
            }

        if not mask:
            rois_dict_brain = {
                roi_name: np.where(roi_mask)[0]
                for roi_name, roi_mask in rois_dict_brain.items()
            }

        return rois_dict_brain

    # --------------------------------------------------
    # Hemispheres
    # --------------------------------------------------
    if isinstance(hemis, str):
        hemis = [hemis]

    if not isinstance(hemis, (list, tuple)):
        raise ValueError(
            "Invalid value for 'hemis'. It should be 'hemi-L', 'hemi-R' or ['hemi-L', 'hemi-R']."
        )

    for hemi in hemis:
        if hemi not in ['hemi-L', 'hemi-R']:
            raise ValueError(
                "Invalid value for 'hemis'. It should be 'hemi-L', 'hemi-R' or ['hemi-L', 'hemi-R']."
            )

    rois_dict_by_hemi = {}

    for hemi in hemis:
        rois_dict_hemi = dict(
            np.load(
                '{}/{}_{}_{}_rois-{}.npz'.format(
                    rois_dir, subject, surf_format, hemi, atlas_name
                )
            )
        )

        if rois is not None:
            rois_dict_hemi = {
                roi_name: roi_mask
                for roi_name, roi_mask in rois_dict_hemi.items()
                if roi_name in rois
            }

        if not mask:
            rois_dict_hemi = {
                roi_name: np.where(roi_mask)[0]
                for roi_name, roi_mask in rois_dict_hemi.items()
            }

        rois_dict_by_hemi[hemi] = rois_dict_hemi

    if len(rois_dict_by_hemi) == 1:
        return rois_dict_by_hemi[hemis[0]]

    return rois_dict_by_hemi

In [28]:
rois_dict

{'mPCS': array([False, False, False, ..., False, False, False], shape=(459846,)),
 'sPCS': array([False, False, False, ..., False, False, False], shape=(459846,)),
 'iPCS': array([False, False, False, ..., False, False, False], shape=(459846,)),
 'sIPS': array([False, False, False, ..., False, False, False], shape=(459846,)),
 'iIPS': array([False, False, False, ..., False, False, False], shape=(459846,)),
 'VO': array([False, False, False, ..., False, False, False], shape=(459846,)),
 'LO': array([False, False, False, ..., False, False, False], shape=(459846,)),
 'hMT+': array([False, False, False, ..., False, False, False], shape=(459846,)),
 'V3AB': array([False, False, False, ..., False, False, False], shape=(459846,)),
 'V3': array([False, False, False, ..., False, False, False], shape=(459846,)),
 'V2': array([ True,  True,  True, ..., False, False, False], shape=(459846,)),
 'V1': array([False, False, False, ..., False, False, False], shape=(459846,))}

In [10]:
cortex.database.default_filestore


'/Users/uriel/disks/meso_shared/RetinoMaps/derivatives/pp_data/cortex/db/'

In [30]:
import numpy as np
rois_idx_L = {roi: np.where(rois_dict[roi])[0] for roi in rois_dict}

In [31]:
rois_idx_L

{'mPCS': array([103034, 103035, 103036, ..., 385000, 459677, 459678], shape=(8061,)),
 'sPCS': array([ 93679,  93680,  93692, ..., 382190, 382201, 382202], shape=(8262,)),
 'iPCS': array([114698, 114710, 114723, ..., 400664, 400665, 400666], shape=(6079,)),
 'sIPS': array([ 12009,  12010,  12013, ..., 305305, 305306, 305326], shape=(9950,)),
 'iIPS': array([  2417,   2418,   2419, ..., 263261, 263275, 263276], shape=(5675,)),
 'VO': array([ 10878,  10879,  10880, ..., 311968, 311969, 459629],
       shape=(12100,)),
 'LO': array([  4684,   4701,   4702, ..., 258225, 258243, 258258], shape=(5487,)),
 'hMT+': array([ 18352,  18378,  18379, ..., 284982, 285001, 285020], shape=(2302,)),
 'V3AB': array([  2216,   2230,   2231, ..., 249424, 249437, 249451], shape=(5683,)),
 'V3': array([   199,    200,    201, ..., 276988, 276989, 276990], shape=(6927,)),
 'V2': array([     0,      1,      2, ..., 276862, 276863, 276907], shape=(7216,)),
 'V1': array([    16,    126,    127, ..., 271894, 271

In [32]:
suat

In [39]:
a = get_rois(subject='sub-01', surf_format='fsnative', atlas_name='drawn', mask=True, rois=None, hemis=['hemi-L','hemi-R'])

In [40]:
a

{'hemi-L': {'mPCS': array([False, False, False, ..., False, False, False], shape=(229831,)),
  'sPCS': array([False, False, False, ..., False, False, False], shape=(229831,)),
  'iPCS': array([False, False, False, ..., False, False, False], shape=(229831,)),
  'sIPS': array([False, False, False, ..., False, False, False], shape=(229831,)),
  'iIPS': array([False, False, False, ..., False, False, False], shape=(229831,)),
  'VO': array([False, False, False, ..., False, False, False], shape=(229831,)),
  'LO': array([False, False, False, ..., False, False, False], shape=(229831,)),
  'hMT+': array([False, False, False, ..., False, False, False], shape=(229831,)),
  'V3AB': array([False, False, False, ..., False, False, False], shape=(229831,)),
  'V3': array([False, False, False, ..., False, False, False], shape=(229831,)),
  'V2': array([ True,  True,  True, ..., False, False, False], shape=(229831,)),
  'V1': array([False, False, False, ..., False, False, False], shape=(229831,))},
 'h

In [41]:
a['hemi-L']

{'mPCS': array([False, False, False, ..., False, False, False], shape=(229831,)),
 'sPCS': array([False, False, False, ..., False, False, False], shape=(229831,)),
 'iPCS': array([False, False, False, ..., False, False, False], shape=(229831,)),
 'sIPS': array([False, False, False, ..., False, False, False], shape=(229831,)),
 'iIPS': array([False, False, False, ..., False, False, False], shape=(229831,)),
 'VO': array([False, False, False, ..., False, False, False], shape=(229831,)),
 'LO': array([False, False, False, ..., False, False, False], shape=(229831,)),
 'hMT+': array([False, False, False, ..., False, False, False], shape=(229831,)),
 'V3AB': array([False, False, False, ..., False, False, False], shape=(229831,)),
 'V3': array([False, False, False, ..., False, False, False], shape=(229831,)),
 'V2': array([ True,  True,  True, ..., False, False, False], shape=(229831,)),
 'V1': array([False, False, False, ..., False, False, False], shape=(229831,))}